encoder transformer层的linear2层（etl2）替换为近似矩阵乘法

In [9]:
# import sys
# sys.path.append('../../../../csi_transformer/src/')
# import backbone
# import image_segmentation
import numpy as np
import os
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [10]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
nbits = 8 # 量化比特数

In [11]:
linear_name = 'etl2'
feedback_bits = 256
linear_name_full = "ex_linear2"
ncodebooks = 32 # max:512
ncentroids = 64

train_sam_num = 500 # 训练集样本数
test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [12]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    linearin_path_train= ''
    linearout_path_train= ''
    linearin_path_test = ''
    linearout_path_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_joined = '/data/hdr/transformer_data/joined'
    dir_train = os.path.join(dir_joined, 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join(dir_joined, 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    linearin_path_train= '%sin_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    y_train = '%s_y_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearout_path_train= '%sout_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearin_path_test = '%sin_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
    linearout_path_test = '%sout_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
else:
    raise NameError("You are running the script in a new computer %s, please define dirs" % host_name)


weightpath = '%s_w_f%i.npy' % (linear_name_full, feedback_bits)
biaspath = '%s_b_f%i.npy' % (linear_name_full, feedback_bits)
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 


In [13]:
dataset_prepare(dir_joined, linear_name_full, feedback_bits, [train_sam_num, test_sam_num], batch_size, S1 = S1)

In [14]:
est3 = mm.estFactory(X_path=linearin_path_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits)

running method:  Mithral
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    62344.23507052845
learn_multisplits(): returning loss:  1880.2048258530028
learn_multisplits(): initial loss:    94893.25950236958
learn_multisplits(): returning loss:  4330.088568210602
learn_multisplits(): initial loss:    1001.0946200168173
learn_multisplits(): returning loss:  317.40761768659837
learn_multisplits(): initial loss:    33843.496185549826
learn_multisplits(): returning loss:  1213.1003481872822
learn_multisplits(): initial loss:    78496.54801032713
learn_multisplits(): returning loss:  4576.129696643808
learn_multisplits(): initial loss:    240839.66908919992
learn_multisplits(): returning loss:  8124.091533660889
learn_multisplits(): initial loss:    2190.252637370293
learn_multisplits(): returning loss:  689.3455305510574
learn_multisplits(): initial loss:    61175.6195721784
learn_multisplits(): returning loss:  1187.3697181977318
learn_multisplits(): initial l

/data/hdr/pq/bolt/experiments/python/clusterize.py:1902: UserWarning: Persisting input arguments took 1.11s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  _learn_mithral_initialization(X, ncodebooks, ncentroids=ncentroids, pq_perm_algo='start', **kwargs)


X_res mse / X mse:  0.004300215
fitting dense lstsq to X_res
  with X_enc:(512000, 32) Y:(512000, 512)
fitted dense lstsq with W:(2048, 512)
X_res mse / X mse after lstsq:  0.100249365


/data/hdr/pq/bolt/experiments/python/vquantizers.py:628: UserWarning: Persisting input arguments took 1.10s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  self.splits_lists, self.centroids = clusterize.learn_mithral(
/data/hdr/pq/bolt/experiments/python/matmul.py:325: UserWarning: Persisting input arguments took 1.24s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  est = _fitted_est_for_hparams(


In [15]:
x_test = np.load(dir_test+'/'+linearin_path_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [16]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last_re.astype(np.float32))
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)

[[ 0.63159144  0.09878757  0.35017315 ...  0.61172235 -0.07011855
   0.55944514]
 [ 0.38110223  0.09878757  0.35017315 ...  0.61172235 -0.07011855
   0.30895597]
 [ 0.63159144  0.09878757  0.35017315 ...  0.61172235 -0.07011855
   0.55944514]
 ...
 [ 0.38110223  0.09878757  0.0996839  ...  0.11074387  0.18037069
   0.30895597]
 [ 0.38110223 -0.15170167  0.35017315 ...  0.11074387 -0.07011855
   0.30895597]
 [ 0.38110223  0.09878757  0.0996839  ...  0.11074387 -0.07011855
   0.05846671]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
